Define your variables (That's the only cells you need to change). Then run the whole notebook

In [1]:
image_path = '/home/tli_project/Desktop/Data/T4/stage2/220219/220219_analysis/220219_P48_N1/220219_P48_N1_clahe.tif'
save_path = '/home/tli_project/Desktop/Data/T4/stage2/220219/220219_analysis/'
save_name = 'SyNRA_4D_220219_P48_mask4'

#N2V parameters
model_name ='n2v_3D_v6'
model_path ='/home/tli_project/Desktop/TLI_preprocessing/Work_folder/N2V/models/'

#Clahe parameters:
clipLimit = 1
kernel_size = (45, 45)

In [2]:
if save_path[-1] != '/':
    save_path += '/'

Load needed packages

In [3]:
import tifffile as tif
import numpy as np
import utils.denoise as denoise
import utils.datautils as datautils
import Neurosetta as neu
import matplotlib.pyplot as plt

2022-04-29 14:23:54.767966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# Read 4D GFP image
image_4D = tif.imread(image_path)

In [5]:
image_4D = denoise.N2V_4D(image_4D, model_name=model_name, model_path=model_path, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name) 

2022-04-29 14:21:06.712638: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-29 14:21:06.713251: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tli_project/miniconda3/envs/preprocessing/lib/python3.8/site-packages/cv2/../../lib64:/app/lib
2022-04-29 14:21:06.713260: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-29 14:21:06.713286: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: l-p0-345
2022-04-29 14:21:06.713289: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: l-p0-345
2022-04-29 14:21:06.713347: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find lib

Loading network weights from 'weights_best.h5'.


applying N2V:   0%|          | 0/71 [00:00<?, ?it/s]

The input image is of type uint16 and will be casted to float32 for prediction.


2022-04-29 14:21:06.987571: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-29 14:21:07.011566: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz
applying N2V:   1%|▏         | 1/71 [01:11<1:22:59, 71.14s/it]

The input image is of type uint16 and will be casted to float32 for prediction.


applying N2V:   3%|▎         | 2/71 [02:21<1:21:03, 70.48s/it]

The input image is of type uint16 and will be casted to float32 for prediction.


In [6]:
# remove further noise pixels before applying clahe, by removing 95% quantil
# threshold = np.quantile(image_4D,0.95)
# # threshold = 4 * image_4D.mean()
# image_4D[image_4D<threshold] = 0
# image_4D[mask == 0] = 0

In [5]:
image_4D = denoise.clahe_4D(image_4D, kernel_size=kernel_size, clipLimit=clipLimit, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name)

applying clahe: 100%|██████████| 71/71 [00:11<00:00,  6.36it/s]


In [8]:
# tif.imwrite(save_path+save_name+'1', image_4D, bigtiff=True, resolution=(1./1, 1./1),
#                 metadata={'spacing': 1, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

In [7]:
mask = denoise.mask_4D(image_4D, return_mask=False, 
                        xy_pixel=0.076, z_pixel=0.4, 
                        sig=1, save=True, 
                        save_path=save_path, 
                        save_file=save_name)

masking the image: 100%|██████████| 71/71 [00:28<00:00,  2.53it/s]


In [14]:
mask.min(), mask.max(), mask.mean(), np.quantile(mask, 0.999)

(0, 54287, 117.13803814904556, 28049.0)

In [15]:
mask[mask>0] = 1
image_4D = image_4D * mask
save_file = save_path+save_name+'_masked.tif'
tif.imwrite(save_file, image_4D, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'TZYX'})

/home/tli_project/.local/lib/python3.8/site-packages/tifffile/tifffile.py:2876: UserWarning: <tifffile.TiffWriter 'SyNRA_4D_220219_P48_masked.tif'> truncating ImageJ file
  warnings.warn(


In [10]:
# # masking the image using neurosetta
# image_4D = neu.Neuron(image_4D)
# mask =  neu.mask_Neuron(image_4D)
# image_4D = image_4D.array
# mask = mask.array
# for i, img in enumerate(image_4D):
#     image_4D[i] = datautils.img_limits(mask[i], limit=img.max())
# image_4D = image_4D.astype('int16')
# datautils.save_image(save_path+'mask2_'+save_name, image_4D)

In [11]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)